In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import time
from scipy.special import expit, sinh, cosh, tanh  # SciPy equivalents


sys.path.insert(0, os.path.abspath('../acsefunctions'))


# Define input values for testing
x_values = np.linspace(-5, 5, 100)  # 100 values from -5 to 5
truncation_levels = [2, 5, 10, 15, 20]  # Different truncation levels


# Prepare a DataFrame to store execution times
execution_times = pd.DataFrame(columns=["Function", "Truncation Level", "Custom Time", "SciPy Time"])

for level in truncation_levels:
    # Time the custom exp function
    custom_exp_times = []
    scipy_exp_times = []
    
    for x in x_values:
        custom_result, custom_time = time_function(exp, x, level)
        scipy_result, scipy_time = time_function(np.exp, x)  # Using NumPy for comparison
        
        custom_exp_times.append(custom_time)
        scipy_exp_times.append(scipy_time)

    execution_times = execution_times.append({
        "Function": "exp",
        "Truncation Level": level,
        "Custom Time": np.mean(custom_exp_times),
        "SciPy Time": np.mean(scipy_exp_times)
    }, ignore_index=True)


# Prepare a DataFrame to store errors
errors = pd.DataFrame(columns=["Function", "Truncation Level", "Max Error"])

for level in truncation_levels:
    for x in x_values:
        custom_result = exp(x, level)
        scipy_result = np.exp(x)  # SciPy equivalent for exp
        
        error = abs(custom_result - scipy_result)
        
        errors = errors.append({
            "Function": "exp",
            "Truncation Level": level,
            "Max Error": np.max(error)
        }, ignore_index=True)

# Repeat for sinh, cosh, and tanh


# Summary DataFrame
summary_df = pd.DataFrame({
    "Function": execution_times['Function'],
    "Average Custom Time": execution_times['Custom Time'],
    "Average SciPy Time": execution_times['SciPy Time'],
    "Max Error": errors.groupby('Function')['Max Error'].max()
}).drop_duplicates()

print(summary_df)


def time_function(func, x, *args):
    start_time = time.time()
    result = func(x, *args)
    end_time = time.time()
    return result, end_time - start_time